In [ ]:
!pip install google-ml-edu==0.1.2 \
    keras~=3.8.0 \
    matplotlib~=3.10.0 \
    numpy~=2.0.0 \
    pandas~=2.2.0 \
    tensorflow~=2.18.0
!pip install streamlit
!pip install tensorflow pandas numpy scikit-learn

In [2]:
import streamlit as st
import io
import keras
from matplotlib import pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [15]:
pd.options.display.float_format = '{:.2f}'.format

In [16]:
crop_data = pd.read_csv("Crop_recommendation.csv")
crop_data.head()

,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.88,82.00,6.50,202.94,rice
1,85,58,41,21.77,80.32,7.04,226.66,rice
2,60,55,44,23.00,82.32,7.84,263.96,rice
3,74,35,40,26.49,80.16,6.98,242.86,rice
4,78,42,42,20.13,81.60,7.63,262.72,rice


In [22]:
numeric_values = crop_data.select_dtypes(include = np.number)
z_mean = numeric_values.mean()
z_std = numeric_values.std()
z_values = (numeric_values - z_mean) / z_std
z_values['label'] = crop_data['label']
z_values.head()


,N,P,K,temperature,humidity,ph,rainfall,label
0,1.07,-0.34,-0.10,-0.94,0.47,0.04,1.81,rice
1,0.93,0.14,-0.14,-0.76,0.40,0.73,2.24,rice
2,0.26,0.05,-0.08,-0.52,0.49,1.77,2.92,rice
3,0.64,-0.56,-0.16,0.17,0.39,0.66,2.54,rice
4,0.74,-0.34,-0.12,-1.08,0.45,1.50,2.90,rice


In [79]:
label_encoder = LabelEncoder()
# Fit the encoder on the crop names
label_encoder.fit(z_values['label_name'])
# Transform the crop names to numerical labels
z_values['label'] = label_encoder.transform(z_values['label_name'])
# Keep the label_name column for reference if needed later
# z_values['label_name'] = crop_data['label']

In [50]:
features = z_values.drop(columns = ['label', 'label_name'])
labels = z_values[['label']]

train_features, temp_features, train_labels, temp_labels = train_test_split(features, labels, test_size=0.4,random_state=42)
test_features, val_features, test_labels, val_labels = train_test_split(temp_features, temp_labels, test_size=0.2, random_state=42)

In [51]:
from tensorflow.keras.utils import to_categorical

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)
val_labels = to_categorical(val_labels)

In [65]:
input_shape = train_features.shape[1]
num_classes = len(z_values['label'].unique())

model = keras.Sequential([
    keras.layers.Input(shape=(input_shape,)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_features,
    train_labels,
    validation_data=(val_features, val_labels),
    epochs=20,
    batch_size=32
)

Epoch 1/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.1425 - loss: 3.0175 - val_accuracy: 0.3636 - val_loss: 2.7037
Epoch 2/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.4004 - loss: 2.5462 - val_accuracy: 0.5000 - val_loss: 2.2175
Epoch 3/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5395 - loss: 2.0412 - val_accuracy: 0.6477 - val_loss: 1.6387
Epoch 4/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6389 - loss: 1.5283 - val_accuracy: 0.7443 - val_loss: 1.1355
Epoch 5/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7873 - loss: 1.0385 - val_accuracy: 0.8636 - val_loss: 0.7848
Epoch 6/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9074 - loss: 0.7077 - val_accuracy: 0.8693 - val_loss: 0.5714
Epoch 7/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9049 - loss: 0.5253 - val_accuracy: 0.8807 - val_loss: 0.4374
Epoch 8/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9273 - loss: 0.4203 - val_accuracy: 0.9091 - val_loss

In [66]:
test_loss, test_accuracy = model.evaluate(test_features, test_labels)
print(f'test accuracy: {test_accuracy}')
print(f'test loss: {test_loss}')

22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9592 - loss: 0.1433 
test accuracy: 0.9588068127632141
test loss: 0.14610829949378967


In [ ]:
predictions = model.predict(test_features)
prediction_labels = np.argmax(predictions, axis = 1)

print(f"predictions: {prediction_labels[:10]}")
print(f"actual: {test_labels[:10]}")

In [71]:
model.save("crop_recommendation_model.keras")
print("my_model.h5")

my_model.h5


In [ ]:
N = float(input("Enter Nitrogen (N): "))
P = float(input("Enter Phosphorus (P): "))
K = float(input("Enter Potassium (K): "))
temperature = float(input("Enter temperature (°C): "))
humidity = float(input("Enter humidity (%): "))
ph = float(input("Enter pH: "))
rainfall = float(input("Enter rainfall (mm): "))

new_data = {
    'N': N,
    'P': P,
    'K': K,
    'temperature': temperature,
    'humidity': humidity,
    'ph': ph,
    'rainfall': rainfall
}

In [80]:
input_df = pd.DataFrame([new_data])
# Apply Z-score standardization using the pre-calculated mean and std
scaled_input = (input_df - z_mean) / z_std
prediction = model.predict(scaled_input)
# Use the already fitted label encoder
predicted_class = label_encoder.inverse_transform(np.argmax(prediction, axis=1))
print(f"Predicted crop: {predicted_class[0]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
Predicted crop: mothbeans
